[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/GeoDream-colab/blob/main/GeoDream_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/GeoDream
%cd /content/GeoDream

!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q ninja lightning==2.0.0 omegaconf jaxtyping controlnet_aux diffusers==0.20.2 trimesh wandb
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl
# !python -c "$(curl -fsSL https://raw.githubusercontent.com/mit-han-lab/torchsparse/master/install.py)"
!pip install -q https://github.com/camenduru/One-2-3-45/releases/download/colab/torchsparse-1.4.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://github.com/camenduru/One-2-3-45/releases/download/colab/inplace_abn-1.1.1.dev9+ge89e592-cp310-cp310-linux_x86_64.whl
!pip install -q git+https://github.com/ashawkey/envlight
!pip install -q git+https://github.com/NVlabs/nvdiffrast
!pip install -q nerfacc==0.5.2 -f https://nerfacc-bucket.s3.us-west-2.amazonaws.com/whl/torch-2.0.0_cu118.html
# !pip install -q git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
!pip install -q rembg open-clip-torch accelerate pyhocon PyMCubes icecream kornia xformers==0.0.20 libigl

!apt -y install -qq aria2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/One-2-3-45/resolve/main/SparseNeuS_demo_v1/exp/lod0/checkpoints/ckpt_215000.pth -d /content/GeoDream/mv-diffusion/One-2-3-45/reconstruction/exp/lod0/checkpoints -o ckpt_215000.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/MVDream/resolve/main/sd-v2.1-base-4view.pt -d /content/GeoDream/mv-diffusion/MVDream/weight -o sd-v2.1-base-4view.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/zero123plus-v1.1/raw/main/inference.py -d /content/GeoDream/mv-diffusion/zero123plus -o inference.py

In [ ]:
%cd /content/GeoDream/mv-diffusion/MVDream
!python t2i_single.py --text "An astronaut riding a horse" --num_frames 4 --camera_elev 15 --camera_azim 180 --camera_azim_span 360
!cp -r remove_bg ../zero123plus/img/remove_bg
%cd /content/GeoDream/mv-diffusion/zero123plus/examples
!python img_to_6mv.py

%cd /content/GeoDream/mv-diffusion/zero123plus/rank_again
!python rank_and_resize.py --project_name 'an_astronaut_riding_a_horse'

%cd /content/GeoDream/mv-diffusion/zero123plus/result
!cp -r /content/GeoDream/mv-diffusion/zero123plus/result/an_astronaut_riding_a_horse /content/GeoDream/mv-diffusion/One-2-3-45/exp
%cd /content/GeoDream/mv-diffusion/One-2-3-45/exp
!cp /content/GeoDream/mv-diffusion/One-2-3-45/exp/pose.json /content/GeoDream/mv-diffusion/One-2-3-45/exp/an_astronaut_riding_a_horse/pose.json

In [ ]:
%cd /content/GeoDream/mv-diffusion/One-2-3-45/reconstruction

!CUDA_VISIBLE_DEVICES=0 python exp_runner_generic_blender_val.py \
                        --specific_dataset_name exp/an_astronaut_riding_a_horse \
                        --mode export_mesh \
                        --conf confs/one2345_lod0_val_demo.conf \
                        --resolution 256

In [ ]:
#Stage 1 (NeuS)
%cd /content/GeoDream
!python launch.py --config configs/geodream-neus.yaml \
                  --train --gpu 0 system.prompt_processor.prompt="an astronaut riding a horse" \
                  system.geometry.init_volume_path="/content/GeoDream/mv-diffusion/One-2-3-45/exp/an_astronaut_riding_a_horse/con_volume_lod_150.pth"
                  # data.width=64 data.height=64

In [ ]:
#Stage 2 (DMTet Geometry Refinement)
!python launch.py --config configs/geodream-dmtet-geometry.yaml --train \
                  system.geometry_convert_from=path/to/stage1/trial/dir/ckpts/last.ckpt --gpu 0 \
                  system.prompt_processor.prompt="an astronaut riding a horse" \
                  system.renderer.context_type=cuda system.geometry_convert_override.isosurface_threshold=0.0

In [ ]:
#Stage 3 (DMTet Texturing)
!python launch.py --config configs/geodream-dmtet-texture.yaml system.geometry.isosurface_resolution=256 \ 
                  --train data.batch_size=1 system.renderer.context_type=cuda --gpu 0 \
                  system.geometry_convert_from=path/to/stage2/trial/dir/ckpts/last.ckpt \ 
                  system.prompt_processor.prompt="an astronaut riding a horse"